# AWS Lambda を MCPify する
## AWS Lambda 関数を Bedrock AgentCore Gateway で安全な MCP ツールに変換する

## 概要
Bedrock AgentCore Gateway は、お客様が既存の AWS Lambda 関数をインフラやホスティングを管理することなく、完全に管理された MCP サーバーに変換する方法を提供します。Gateway はこれらすべてのツールに対して統一された Model Context Protocol (MCP) インターフェースを提供します。Gateway は、ターゲットリソースへの着信リクエストと送信接続の両方に対して安全なアクセス制御を確保するために、二重認証モデルを採用しています。このフレームワークは 2 つの主要コンポーネントで構成されています：インバウンド認証（ゲートウェイターゲットへのアクセスを試みるユーザーを検証して承認する）と、アウトバウンド認証（認証されたユーザーに代わってゲートウェイがバックエンドリソースに安全に接続できるようにする）です。Gateway はアウトバウンド認証のために AWS Lambda 関数への呼び出しを承認するために IAM ロールを使用します。

![仕組み](images/lambda-iam-gateway.png)

### チュートリアルの詳細

| 情報                 | 詳細                                                     |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ | インタラクティブ                                         |
| AgentCore コンポーネント | AgentCore Gateway, AgentCore Identity                     |
| エージェントフレームワーク | Strands Agents                                            |
| Gateway ターゲットタイプ | AWS Lambda                                                |
| インバウンド認証 IdP | Amazon Cognito                                            |
| アウトバウンド認証   | AWS IAM                                                   |
| LLM モデル           | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| チュートリアルコンポーネント | AgentCore Gateway の作成と AgentCore Gateway の呼び出し |
| チュートリアル分野   | 分野横断的                                               |
| 例の複雑さ           | 簡単                                                     |
| 使用する SDK         | boto3                                                     |

チュートリアルの最初のパートでは、いくつかの AmazonCore Gateway ターゲットを作成します

### チュートリアルのアーキテクチャ
このチュートリアルでは、AWS Lambda 関数で定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。
デモンストレーションの目的で、Amazon Bedrock モデルを使用する Strands Agent を使用します。
この例では、get_order と update_order という 2 つのツールを持つ非常にシンプルなエージェントを使用します。

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Jupyter notebook（Python カーネル）
* uv
* AWS 認証情報
* Amazon Cognito

## AgentCore Gateway への受信リクエストの認証設定
AgentCore Gateway は、インバウンドおよびアウトバウンド認証を通じて安全な接続を提供します。インバウンド認証では、AgentCore Gateway は呼び出し時に渡される OAuth トークンを分析し、ゲートウェイ内のツールへのアクセスを許可または拒否するかを決定します。ツールが外部リソースにアクセスする必要がある場合、AgentCore Gateway は API キー、IAM、または OAuth トークンを介したアウトバウンド認証を使用して、外部リソースへのアクセスを許可または拒否することができます。

インバウンド認証フローでは、エージェントまたは MCP クライアントが、OAuth アクセストークン（ユーザーの IdP から生成された）を追加して AgentCore Gateway 内の MCP ツールを呼び出します。AgentCore Gateway はその後、OAuth アクセストークンを検証し、インバウンド認証を実行します。

AgentCore Gateway で実行されているツールが外部リソースにアクセスする必要がある場合、OAuth はゲートウェイターゲットのリソース認証情報プロバイダーを使用して、下流リソースの認証情報を取得します。AgentCore Gateway は、下流 API へのアクセスを得るために、認証情報を呼び出し元に渡します。

In [ ]:
# 最新の botocore と boto3 ライブラリをダウンロードしていることを確認してください。
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

In [ ]:
# Amazon SageMaker ノートブックを使用していない場合は AWS 認証情報を設定する
import os
os.environ['AWS_ACCESS_KEY_ID'] = '' # アクセスキーを設定する
os.environ['AWS_SECRET_ACCESS_KEY'] = '' # シークレットを設定する
os.environ['AWS_PROFILE'] = 'cline2'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1' # AWS リージョンを設定する

In [ ]:
import os
import sys

# 現在のスクリプトのディレクトリを取得する
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__ が定義されていない場合のフォールバック（例：Jupyter）

# utils.py があるディレクトリ（1 レベル上）に移動する
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# 今、 utils をインポートできます
import utils

In [ ]:
import time

#### AWS Lambda 関数のサンプルを作成し、MCP ツールに変換する
lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")

if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("Lambda function created with ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda function creation failed with message: ", lambda_resp['lambda_function_arn'])

# IAMロールの伝播を待つ
print("Waiting for IAM role to propagate...")
time.sleep(15)  # 15秒待機

# Lambda関数を作成
lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")

if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("Lambda function created with ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda function creation failed with message: ", lambda_resp['lambda_function_arn'])

In [ ]:
#### Gateway が引き受ける IAM ロールを作成する
import utils
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway へのインバウンド認証のための Amazon Cognito プールの作成

In [ ]:
# Cognito ユーザープールの作成
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# ディスカバリー URL の取得
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# Amazon Cognito オーソライザーを使用したインバウンド認可のためのゲートウェイの作成

In [ ]:
# CMK なしで Cognito オーソライザーを使用して CreateGateway。前のステップで作成した Cognito ユーザープールを使用する
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # クライアントは Cognito で設定された ClientId と一致する必要があります。例： 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='TestGWforLambda',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM ロールは Gateway の作成・一覧表示・取得・削除の権限を持っている必要があります
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with AWS Lambda target type'
)
print(create_response)
# GatewayTarget 作成に使用される GatewayID を取得する
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# AWS Lambda ターゲットを作成し MCP ツールに変換する

In [ ]:
# AWS Lambda 関数の ARN を以下に置き換えてください
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_resp['lambda_function_arn'], # AWS Lambda 関数 ARN をここに置き換えてください
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_tool",
                        "description": "tool to get the order",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    },                    
                    {
                        "name": "update_order_tool",
                        "description": "tool to update the orderId",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    }
                ]
            }
        }
    }
}

credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='LambdaUsingSDK'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target using SDK',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装する Bedrock AgentCore Gateway を通じて AWS ツールとシームレスに統合されています。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

その核心部分として、Bedrock AgentCore Gateway は基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠のゲートウェイとして機能します。これらの API により、MCP 準拠のクライアントや SDK は、安全で標準化された方法で利用可能なツールを発見し、対話することができます。Strands エージェントが AWS サービスにアクセスする必要がある場合、これらの MCP 標準化されたエンドポイントを使用してゲートウェイと通信します。

ゲートウェイの実装は (MCP 認証仕様)[https://modelcontextprotocol.org/specification/draft/basic/authorization] に厳密に準拠しており、堅牢なセキュリティとアクセス制御を確保しています。これは、Strands エージェントによるすべてのツール呼び出しが認証ステップを経ることを意味し、強力な機能を提供しながらセキュリティを維持します。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを発見し、次に InvokeTools を使用して特定のアクションを実行します。ゲートウェイは必要なすべてのセキュリティ検証、プロトコル変換、サービス連携を処理し、プロセス全体をシームレスかつ安全にします。

このアーキテクチャアプローチにより、MCP 仕様を実装するあらゆるクライアントや SDK がゲートウェイを通じて AWS サービスと対話できるようになり、AI エージェント統合のための汎用性と将来性のあるソリューションとなっています。

![Strands エージェントがゲートウェイを呼び出す](images/strands-lambda-gateway.png)

In [ ]:
uv_install("mcp[cli]", "strands-agents")

# インバウンド認証のために Amazon Cognito からアクセストークンをリクエストする

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# AWS Lambda の MCP ツールを Bedrock AgentCore Gateway を使用して呼び出す Strands エージェント

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentials で設定された IAM 認証情報は Bedrock モデルへのアクセス権を持っている必要があります
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging

# ルートの strands ロガーを設定します。問題をデバッグしている場合は、DEBUG に変更してください。
logging.getLogger("strands").setLevel(logging.INFO)

# ログを確認するためのハンドラーを追加する
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # モデルとツールを使用したエージェントの作成
    agent = Agent(model=yourmodel, tools=tools) ## あなたは どのモデルでも 好きなものに 置き換えることができます
    print(f"Tools loaded in the agent are {agent.tool_names}")
    
    # 修正: tool_config の代わりに tool_registry を使用
    print(f"Tools registry in the agent: {agent.tool_registry}")
    
    # サンプルプロンプトでエージェントを呼び出します。これは MCP listTools のみを呼び出し、LLM がアクセスできるツールのリストを取得します。以下は実際にはどのツールも呼び出しません。
    agent("Hi , can you list all tools available to you")
    # エージェントをサンプルプロンプトで呼び出し、ツールを実行して応答を表示する
    agent("Check the order status for order id 123 and show me the exact response from the tool")
    # MCP ツールを明示的に呼び出します。MCP ツール名と引数は、AWS Lambda 関数または OpenAPI/Smithy API と一致する必要があります
    result = client.call_tool_sync(
    tool_use_id="get-order-id-123-call-1", # あなたはこれを一意の識別子に置き換えることができます。
    name=targetname+"___get_order_tool", # これは AWS Lambda ターゲットタイプに基づくツール名です。ターゲット名に応じて変更されます
    arguments={"orderId": "123"}
    )
    # ログを確認するためのハンドラーを追加する
    print(f"Tool Call result: {result['content'][0]['text']}")


**問題: 以下のセルを実行中に下記のエラーが表示される場合、pydantic と pydantic-core のバージョン間に互換性がないことを示しています。**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```

**解決方法は？**

互換性のある pydantic==2.7.2 と pydantic-core 2.27.2 をインストールする必要があります。インストール完了後、カーネルを再起動してください。

# クリーンアップ

IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、S3 バケットなどの追加リソースも作成されており、クリーンアップの一環として手動で削除する必要があるかもしれません。これは実行する例によって異なります。

## ゲートウェイの削除 (オプション)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)